## Feature Extraction

**Packages**

In [1]:
import os
import pandas as pd
import numpy as np

import torch
import torchvision.models as tmodels
from torch.autograd import Variable
from torchvision import transforms

import cv2
from PIL import Image
import h5py
from tqdm import tqdm
from time import time

**Variables**

In [2]:
EXTRACT_FREQUENCY = 15
BATCH_SIZE = 16

args = {'use_cpu':True}

##### )) Paths

In [3]:

datasets_path='../data'
public_dataset_path=datasets_path+'/Public datasets'
tvsum_data = public_dataset_path+'/ydata-tvsum50-v1_1'
summe_data = public_dataset_path+'/SUMMe'
custom_data = datasets_path+'/Custom data'


features_path = 'extracted_features'
normalFt_path = features_path+'/normal'
hashbasedFt_path = features_path+'/hashbased'

##### )) GoogLeNet Model

In [4]:
googlenet = tmodels.googlenet(pretrained=True)
googlenet = torch.nn.Sequential(*list(googlenet.children())[:-2])
# googlenet.eval()

C:\Users\Msc 2\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\Msc 2\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
device = 'cuda' if torch.cuda.is_available() and not use_cpu else 'cpu'
googlenet = googlenet.to(device)

##### )) Frame preprocessing

In [6]:
  
def preprocess(frame):
    tr = transforms.Compose([
        transforms.Resize(224),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # BGR to RGB
    im = Image.fromarray(im) # cv2 to PIL
    im = tr(im)
    fr = np.array(im)
    
    return fr


### Extraction Modules

##### )) Hashed based features

In [7]:
def dhash(img):
    img=cv2.resize(img,(9,8),interpolation=cv2.INTER_CUBIC)
    # gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

    dhash_str = ''
    for i in range(8):
        for j in range(8):
            if gray[i, j] > gray[i, j + 1]:
                dhash_str = dhash_str + '1'
            else:
                dhash_str = dhash_str + '0'
    result = ''
    for i in range(0, len(dhash_str), 4):
        result += ''.join('%x' % int(dhash_str[i: i + 4], 2))
    # print(result)
    return result

def hamming(s1, s2):
    assert len(s1) == len(s2)
    return sum([ch1 != ch2 for ch1, ch2 in zip(s1, s2)])

In [8]:
def hash_gen_features(path, threshold):
    cap = cv2.VideoCapture(path)
    frames=[]
    video_features = []
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    count = 0
    skip_count = 0
    
    nframes = frame_count//EXTRACT_FREQUENCY * EXTRACT_FREQUENCY
    
    with torch.no_grad():
        base = None
        hash1 = None
        while cap.isOpened():
            # Capture frame-by-frame
            ret, fr = cap.read()
            if ret is False:
                break
            count += 1
            
            frame = preprocess(fr)

            if count % EXTRACT_FREQUENCY == 0:
                hash2=dhash(fr)
                if hash1 is not None:
                    dist = hamming(hash1,hash2)
                if base is None or dist > threshold:
                    base = fr
                    hash1 = hash2
                    frames.append(frame)
                else:
                    skip_count += 1
                    frames.append(frame)
                    
                if (len(frames) == BATCH_SIZE) or (count == nframes and len(frames) > 0):
                    batch = np.array(frames)
                    if args['use_cpu']:
                        variable = Variable(torch.from_numpy(batch).float())
                        feature = googlenet(variable).detach().numpy()
                    else:
                        variable = Variable(torch.from_numpy(batch).float()).cuda()
                        feature = googlenet(variable).cpu().detach().numpy()
                        
                    video_features.extend(feature)
                    frames.clear()
                    

    cap.release()
    video_features = np.squeeze(np.array(video_features))
    return video_features, frame_count, fps

##### )) Normal features

In [9]:
def gen_features(path):
    cap = cv2.VideoCapture(path)
    count = 0
    frames = []
    video_features = []
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    nframes = frame_count//EXTRACT_FREQUENCY * EXTRACT_FREQUENCY
    
    with torch.no_grad():
        
        while cap.isOpened():
            ret, fr = cap.read()
            if ret is False:
                break

            fr = preprocess(fr)
            
            count += 1
            if count % EXTRACT_FREQUENCY == 0:
                # frames.append(np.rollaxis(fr, 2))
                frames.append(fr)
                
                
            if (len(frames) == BATCH_SIZE) or (count == nframes and len(frames) > 0):
                batch = np.array(frames)
                # print(batch.shape)
                if args['use_cpu']:
                    variable = Variable(torch.from_numpy(batch).float())
                    feature = googlenet(variable).detach().numpy()
                else:
                    variable = Variable(torch.from_numpy(batch).float()).cuda()
                    feature = googlenet(variable).cpu().detach().numpy()
                video_features.extend(feature)
                frames.clear()
    
            
    cap.release()

    video_features = np.squeeze(np.array(video_features))
    return video_features, frame_count, fps

##### )) Feature extraction and storage

In [10]:
def extract_features(videos_path, h5output_path, method=None, names=None):
    h5_file = h5output_path
    # st = time()

    with h5py.File(h5_file, 'w') as f:
        files = os.listdir(videos_path)
        cnt = 0
        
        pbar = tqdm(total = len(files), position=0, leave=True)
        for file in files:
            cnt += 1
            path = videos_path + "/" + file
            
            if method == 'hashbased':
                video_features, fcnt, fps = hash_gen_features(path,4.0)
            else:
                video_features, fcnt, fps = gen_features(path)

            video_length = fcnt/fps
            
            if names:
                video_ = 'video_'+str(names.index(file.split(".")[0])+1)
            else:
                video_ = 'video_'+str(cnt)

            print(cnt,'.', video_, "no. of frames=", fcnt, "Output dimension=", video_features.shape)

            f.create_dataset(video_ + '/n_frames', data=int(fcnt))
            f.create_dataset(video_ + '/fps', data=int(fps))
            f.create_dataset(video_ + '/features', data=video_features)
            picks = np.arange(0, video_features.shape[0]) * EXTRACT_FREQUENCY
            f.create_dataset(video_ + '/picks', data=picks)
            f.create_dataset(video_+'/duration', data=video_length)
            f.create_dataset(video_+'/video_name', data=str(file))
            pbar.update(1)


### TVSum feature extraction

##### )) TVSumm info

##### )) Videos

##### )) Extraction

In [14]:
# vnames = tvsum_info['video_id'].tolist()
# extract_features(videos_path=tvsum_data+'/video', h5output_path=normalFt_path+'/TVSum.h5', names=vnames)

  2%|▏         | 1/50 [00:58<47:32, 58.21s/it]

1 . video_50 no. of frames= 6912.0 Output dimension= (460, 1024)


  4%|▍         | 2/50 [01:28<33:16, 41.59s/it]

2 . video_13 no. of frames= 3532.0 Output dimension= (235, 1024)


  6%|▌         | 3/50 [02:16<35:08, 44.86s/it]

...

48 . video_18 no. of frames= 9731.0 Output dimension= (648, 1024)


 98%|█████████▊| 49/50 [46:21<01:02, 62.34s/it]

49 . video_28 no. of frames= 8281.0 Output dimension= (552, 1024)


100%|██████████| 50/50 [46:59<00:00, 56.39s/it]

50 . video_35 no. of frames= 4463.0 Output dimension= (297, 1024)


In [ ]:
# extract_features(videos_path=tvsum_data+'/video', h5output_path=hashbasedFt_path+'/TVSum.h5',method='hashbased', names=vnames)

---------------------------

##### *H5File metadata*